In [2]:
import geopandas as gpd
from sqlalchemy import create_engine

/data/mutakabbir/Lightning/venv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
canada_subdivisions = "/data/mutakabbir/Lightning/data/canada/lfed000b21a_e.zip"

In [4]:
DATABASE_TYPE="postgresql"
USERNAME="mutakabbir"
PASSWORD="lightning"
HOST="localhost"
PORT=5432
DATABASE_NAME="postgres"
TABLE_NAME="subdivision_geo"
engine = create_engine(f"{DATABASE_TYPE}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}")

In [5]:
%%time
subdivisions_gpd = gpd.read_file(canada_subdivisions)
subdivisions_gpd = subdivisions_gpd.to_crs(4326)

CPU times: user 16.8 s, sys: 1.81 s, total: 18.6 s
Wall time: 18.7 s


In [6]:
subdivisions_gpd['FEDUID'] = subdivisions_gpd['FEDUID'].astype(int)

In [7]:
subdivisions_gpd.set_index('FEDUID', inplace=True)

In [8]:
subdivisions_gpd.head()

,DGUID,FEDNAME,FEDENAME,FEDFNAME,LANDAREA,PRUID,geometry
FEDUID,,,,,,,
10001,2013A000410001,Avalon,Avalon,Avalon,6357.2457,10,"MULTIPOLYGON (((-53.44465 46.67495, -53.44474 ..."
10002,2013A000410002,Bonavista--Burin--Trinity,Bonavista--Burin--Trinity,Bonavista--Burin--Trinity,16529.4593,10,"MULTIPOLYGON (((-53.77539 48.71901, -53.77552 ..."
10003,2013A000410003,Coast of Bays--Central--Notre Dame,Coast of Bays--Central--Notre Dame,Coast of Bays--Central--Notre Dame,38674.0098,10,"MULTIPOLYGON (((-55.51195 49.45222, -55.51213 ..."
10004,2013A000410004,Labrador,Labrador,Labrador,258539.4032,10,"MULTIPOLYGON (((-55.60642 53.43093, -55.60651 ..."
10005,2013A000410005,Long Range Mountains,Long Range Mountains,Long Range Mountains,37319.1682,10,"MULTIPOLYGON (((-58.32368 49.25135, -58.32372 ..."


In [9]:
try:
    subdivisions_gpd.to_postgis(name=TABLE_NAME, con=engine, if_exists='replace', index=False)  
except Exception as e:
    print(e)
